In [1]:
import geopandas as gpd
df = gpd.read_file('/data/acker/ALA/paper2/all_variables.shp')
df = df.rename(columns={'monitor__1':'monitor_coverage_pct', 'classifica':'classification', 'Design Val':'Design Value', 'monitor_co':'monitor_count',
                        'fire_regio':'fire_region', 'mountain_r':'mountain_region', 'desert_reg':'desert_region', 'urban_cate':'urban_category'})
#upload shapefile of U.S. counties
counties = gpd.read_file('/data/acker/shapefiles/cb_2020_us_county_500k.shp')
# List of state abbreviations for CONUS, Alaska (AK), and Hawaii (HI)
states_to_include = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'DC',
    'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS',
    'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
    'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV',
    'WI', 'WY'
]

# Filter counties to only include rows where STUSPS is in the specified list
counties_conus = counties[counties['STUSPS'].isin(states_to_include)]

# Display the filtered counties
counties_conus = counties_conus.drop(["COUNTYNS", 'NAMELSAD', 'LSAD', 'ALAND', 'AWATER', 'AFFGEOID'], axis=1)
counties_conus = counties_conus.to_crs(df.crs)
df = df.drop(columns='geometry')


# Merge with both GEOID and geometry
new = df.merge(counties_conus[['GEOID', 'geometry']], on='GEOID', how='inner')

# Recast as GeoDataFrame
ranked_correct = gpd.GeoDataFrame(new, geometry='geometry', crs=counties_conus.crs)
df = ranked_correct
df

,GEOID,PM25_90th,Design Value,classification,diff,abs_diff,monitor_count,cdv_bin,monitor_coverage_pct,dist_km,size,fire_region,mountain_region,desert_region,urban_category,geometry
0,02090,14.333333,12.1,TP,2.233333,2.233333,3,>10,0.008092,85.118385,large,Fire,Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-148.66326 64.59079, -148.64821 64.5..."
1,06063,14.200000,14.0,TP,0.200000,0.200000,1,>10,0.013591,60.606568,large,Fire,Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-121.49703 40.43702, -121.49487 40.4..."
2,06107,12.933334,15.7,TP,-2.766666,2.766666,1,>10,0.007784,100.833209,large,Fire,Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-119.56647 36.49434, -119.56366 36.4..."
3,06023,12.633334,6.9,FP,5.733334,5.733334,1,<7,0.009894,28.714886,large,Fire,Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-124.4086 40.4432, -124.39664 40.462..."
4,06029,12.599999,16.2,TP,-3.600001,3.600001,5,>10,0.023405,82.031806,large,Fire,Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-120.19437 35.78936, -120.00308 35.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,02020,4.133333,5.1,TN,-0.966667,0.966667,1,<7,0.020235,66.108117,small,Non-Fire,Mountain,Non-Desert,Non-Urban (<50%),"MULTIPOLYGON (((-150.07348 61.15834, -150.0691..."
532,02110,3.900000,4.6,TN,-0.700000,0.700000,1,<7,0.018594,73.544905,large,Non-Fire,Mountain,Non-Desert,Non-Urban (<50%),"MULTIPOLYGON (((-134.66932 58.33327, -134.6675..."
533,15001,3.633333,4.4,TN,-0.766667,0.766667,5,<7,0.054825,81.864071,large,Fire,Non-Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-156.06147 19.72813, -156.06076 19.7..."
534,15009,3.556667,4.0,TN,-0.443333,0.443333,1,<7,0.035920,78.820617,small,Fire,Non-Mountain,Non-Desert,Non-Urban (<50%),"MULTIPOLYGON (((-156.69742 20.91637, -156.6957..."


In [4]:
removed = gpd.read_file('/data/acker/ALA/paper2/GL_removed_pixels_2021-2023_grids.shp')
rem_counties = gpd.sjoin(removed, df, how='inner', predicate='intersects')
total_removed_per_county = rem_counties.groupby('GEOID').size().reset_index(name='removed_grids')
final_df_1 = df.merge(total_removed_per_county, on='GEOID', how='left')
final_df_1['removed_grids'] = final_df_1['removed_grids'].fillna(0).astype(int)
final_df_1

,GEOID,PM25_90th,Design Value,classification,diff,abs_diff,monitor_count,cdv_bin,monitor_coverage_pct,dist_km,size,fire_region,mountain_region,desert_region,urban_category,geometry,removed_grids
0,02090,14.333333,12.1,TP,2.233333,2.233333,3,>10,0.008092,85.118385,large,Fire,Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-148.66326 64.59079, -148.64821 64.5...",4254
1,06063,14.200000,14.0,TP,0.200000,0.200000,1,>10,0.013591,60.606568,large,Fire,Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-121.49703 40.43702, -121.49487 40.4...",636
2,06107,12.933334,15.7,TP,-2.766666,2.766666,1,>10,0.007784,100.833209,large,Fire,Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-119.56647 36.49434, -119.56366 36.4...",1365
3,06023,12.633334,6.9,FP,5.733334,5.733334,1,<7,0.009894,28.714886,large,Fire,Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-124.4086 40.4432, -124.39664 40.462...",680
4,06029,12.599999,16.2,TP,-3.600001,3.600001,5,>10,0.023405,82.031806,large,Fire,Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-120.19437 35.78936, -120.00308 35.7...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,02020,4.133333,5.1,TN,-0.966667,0.966667,1,<7,0.020235,66.108117,small,Non-Fire,Mountain,Non-Desert,Non-Urban (<50%),"MULTIPOLYGON (((-150.07348 61.15834, -150.0691...",3534
532,02110,3.900000,4.6,TN,-0.700000,0.700000,1,<7,0.018594,73.544905,large,Non-Fire,Mountain,Non-Desert,Non-Urban (<50%),"MULTIPOLYGON (((-134.66932 58.33327, -134.6675...",7984
533,15001,3.633333,4.4,TN,-0.766667,0.766667,5,<7,0.054825,81.864071,large,Fire,Non-Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-156.06147 19.72813, -156.06076 19.7...",72
534,15009,3.556667,4.0,TN,-0.443333,0.443333,1,<7,0.035920,78.820617,small,Fire,Non-Mountain,Non-Desert,Non-Urban (<50%),"MULTIPOLYGON (((-156.69742 20.91637, -156.6957...",141


In [5]:
disagreement = final_df_1[(final_df_1['classification'] == 'FP') | (final_df_1['classification'] == 'FN')]
agreement = final_df_1[(final_df_1['classification'] == 'TP') | (final_df_1['classification'] == 'TN')]

In [6]:
final_df_1.columns

Index(['GEOID', 'PM25_90th', 'Design Value', 'classification', 'diff',
       'abs_diff', 'monitor_count', 'cdv_bin', 'monitor_coverage_pct',
       'dist_km', 'size', 'fire_region', 'mountain_region', 'desert_region',
       'urban_category', 'geometry', 'removed_grids'],
      dtype='object')

In [7]:
# Split into groups
all_df = final_df_1
disagreement_df = final_df_1[final_df_1['classification'].isin(['FP', 'FN'])]
agreement_df = final_df_1[final_df_1['classification'].isin(['TP', 'TN'])]

# Function to compute means for each geography
def mean_removed_grids(df, geography_col, category_name):
    with_category = df[df[geography_col] == category_name]['removed_grids'].mean()
    without_category = df[df[geography_col] != category_name]['removed_grids'].mean()
    return with_category, without_category

# List of geography columns and categories
geographies = [
    ('fire_region', 'Fire'),
    ('mountain_region', 'Mountain'),
    ('desert_region', 'Desert'),
    ('urban_category', 'Non-Urban (<50%)')
]

# Store results
results = {}

for name, group_df in zip(['All Counties', 'Disagreement', 'Agreement'], [all_df, disagreement_df, agreement_df]):
    group_results = {}
    for geo_col, geo_val in geographies:
        with_cat, without_cat = mean_removed_grids(group_df, geo_col, geo_val)
        group_results[geo_val] = {
            'With Category': with_cat,
            'Without Category': without_cat
        }
    results[name] = group_results

# Display results
import pprint
pprint.pprint(results)


{'Agreement': {'Desert': {'With Category': 2486.0,
                          'Without Category': 264.65501165501166},
               'Fire': {'With Category': 454.95412844036696,
                        'Without Category': 269.1121212121212},
               'Mountain': {'With Category': 488.0808080808081,
                            'Without Category': 173.2655601659751},
               'Non-Urban (<50%)': {'With Category': 351.17054263565893,
                                    'Without Category': 47.96153846153846}},
 'All Counties': {'Desert': {'With Category': 1889.857142857143,
                             'Without Category': 226.00574712643677},
                  'Fire': {'With Category': 380.294964028777,
                           'Without Category': 230.65994962216624},
                  'Mountain': {'With Category': 423.2911392405063,
                               'Without Category': 147.5351170568562},
                  'Non-Urban (<50%)': {'With Category': 304.295258620689